In [1]:
from qiskit import quantum_info,transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
from qiskit.quantum_info import Operator
from qiskit_ibm_runtime import Estimator, Session, QiskitRuntimeService, Options, Sampler

In [7]:

def Shat(A,T,G): 
    
    t = np.tanh(T/4 + G/4)
    tt = np.tanh(T/4 - G/4)
    Shatf = np.array([[1-t*tt,0,0,t+tt],[0,1+t*tt,t-tt,0],[0,-t+tt,1+t*tt,0],[-t-tt,0,0,1-t*tt]],dtype=complex)
    Shatg = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,-1]],dtype=complex)
    f = -(1j/4)*A*(2*np.cosh(T/2)+2*np.cosh(G/2))*(1/np.sinh(T/2))*(1/np.cosh(T/2))
    Shat = f*Shatf+Shatg
    Nalpha = 1+2*A**(2)*(np.cosh(T)/np.sinh(T))*(1/np.sinh(T))+2*A**(2)*np.cosh(G)*(1/np.sinh(T))**(2)
    Shat = (-1/np.sqrt(Nalpha))*Shat    #
    return Shat

def K(T):
    
    K2 = Ur = np.zeros((2,2), dtype=complex)
    
    coef1 = np.cos(np.pi/4 + 1j*T/2)
    coef2 = np.cos(np.pi/4 - 1j*T/2)
    
    K2[0,0]=coef1
    K2[1,1]=coef2
    K2 = 1/np.sqrt(np.cosh(T)/2)*K2      #
    
    return K2


def Na(A,T,G):
    return 1+2*A**(2)*(np.cosh(T)/np.sinh(T))*(1/np.sinh(T))+2*A**(2)*np.cosh(G)*(1/np.sinh(T))**(2)

def Nk(T):
    return (np.cosh(T/2))/2

basis_gates=['ecr', 'id', 'rz', 'sx', 'x']

In [3]:
def x(t):
    return np.exp(t/4)

def y(g):
    return np.exp(g/4)

def Qplus(t,g):
    Ql12 = x(t)*y(g)*X(1,4)+(1/(x(t)*y(g)))*Z(1,4)@X(2,4)
    Ql34 = x(t)*y(g)*Z(1,4)@Z(2,4)@X(3,4)+(1/(x(t)*y(g)))*Z(1,4)@Z(2,4)@Z(3,4)@X(4,4)
    return Ql12+Ql34

def Qminus(t,g):
    Qr12 = (y(g)/x(t))*Y(1,4)+(x(t)/y(g))*Z(1,4)@Y(2,4)
    Qr34 = (y(g)/x(t))*Z(1,4)@Z(2,4)@Y(3,4)+(x(t)/y(g))*Z(1,4)@Z(2,4)@Z(3,4)@Y(4,4)
    return Qr12+Qr34

def X(n,L):
    X = np.array([[0,1],[1,0]])
    return (np.kron(np.eye(2**(n-1)),np.kron(X,np.eye(2**(L-n)))))
def Y(n,L):
    Y = np.array([[0,-1j],[1j,0]])
    return (np.kron(np.eye(2**(n-1)),np.kron(Y,np.eye(2**(L-n)))))    
def Z(n,L):
    Z = np.array([[1,0],[0,-1]])
    return (np.kron(np.eye(2**(n-1)),np.kron(Z,np.eye(2**(L-n)))))    

In [9]:
n=4;A=1;G=1;T=1

qa = QuantumRegister(4, name='qa')
qb = QuantumRegister(4, name='qb')
anc = QuantumRegister(1, name='ancilla')
c = ClassicalRegister(1, name='c')
qc = QuantumCircuit(qa, qb,anc,c)
qc.h(8)
qc.barrier(range(2*n+1))
l=0


#local gate way (MUCH CHEAPER IN TRANSPILATION), first half
qc.unitary(Shat(A,T,G),[0,1],label='S12')
qc.unitary(Shat(A,T,G),[2,3],label='S34')
qc.unitary(K(-T/2),[0],label=r'$K(-\theta/2)$')
qc.unitary(Shat(A,T,G),[1,2],label='S23')
qc.unitary(K(T/2),[3],label=r'$K(\theta/2)$')
qc.unitary(Shat(A,T,0),[0,1],label='S12')
qc.unitary(Shat(A,T,0),[2,3],label='S34')
qc.unitary(K(-T/2),[0],label=r'$K(-\theta/2)$')
qc.unitary(Shat(A,T,-G),[1,2],label='S23')
qc.unitary(K(T/2),[3],label=r'$K(\theta/2)$')
qc.unitary(Shat(A,T,-G),[0,1],label='S12')
qc.unitary(Shat(A,T,-G),[2,3],label='S34')
qc.unitary(K(-T/2),[0],label=r'$K(-\theta/2)$')
qc.unitary(Shat(A,T,-G),[1,2],label='S23')
qc.unitary(K(T/2),[3],label=r'$K(\theta/2)$')
qc.unitary(Shat(A,T,0),[0,1],label='S12')
qc.unitary(Shat(A,T,0),[2,3],label='S34')
qc.unitary(K(-T/2),[0],label=r'$K(-\theta/2)$')
qc.unitary(Shat(A,T,G),[1,2],label='S23')
qc.unitary(K(T/2),[3],label=r'$K(\theta/2)$')
#second half
qc.unitary(Shat(A,T,G),[4,5],label='S12B')
qc.unitary(Shat(A,T,G),[6,7],label='S34B')
qc.unitary(K(-T/2),[4],label=r'$K(-\theta/2)B$')
qc.unitary(Shat(A,T,G),[5,6],label='S23B')
qc.unitary(K(T/2),[7],label=r'$K(\theta/2)B$')
qc.unitary(Shat(A,T,0),[4,5],label='S12B')
qc.unitary(Shat(A,T,0),[6,7],label='S34B')
qc.unitary(K(-T/2),[4],label=r'$K(-\theta/2)B$')
qc.unitary(Shat(A,T,-G),[5,6],label='S23B')
qc.unitary(K(T/2),[7],label=r'$K(\theta/2)B$')
qc.unitary(Shat(A,T,-G),[4,5],label='S12B')
qc.unitary(Shat(A,T,-G),[6,7],label='S34B')
qc.unitary(K(-T/2),[4],label=r'$K(-\theta/2)B$')
qc.unitary(Shat(A,T,-G),[5,6],label='S23B')
qc.unitary(K(T/2),[7],label=r'$K(\theta/2)B$')
qc.unitary(Shat(A,T,0),[4,5],label='S12B')
qc.unitary(Shat(A,T,0),[6,7],label='S34B')
qc.unitary(K(-T/2),[4],label=r'$K(-\theta/2)B$')
qc.unitary(Shat(A,T,G),[5,6],label='S23B')
qc.unitary(K(T/2),[7],label=r'$K(\theta/2)B$')
qc.barrier(range(2*n+1))



op = Operator(qc)
op=op.data
qc.cswap(8,0,4)
qc.cswap(8,1,5)
qc.barrier(range(2*n+1))
qc.h(8)
qc.measure(8,0)
qc = transpile(qc, basis_gates=basis_gates, optimization_level=1)
qc.draw(output='mpl')
qc.count_ops()

OrderedDict([('rz', 320),
             ('sx', 220),
             ('ecr', 64),
             ('x', 12),
             ('barrier', 3),
             ('measure', 1)])

In [10]:
N1=320+220+12
N2=64
Nr=1
datapoints=len(np.arange(0.1,3.8,0.05))
shots = 16e3
one_time=150e-9
two_time=350e-9
readout=730e-9


estimated_time=(one_time*N1+two_time*N2+readout*Nr)*datapoints*shots
print(estimated_time/60)

2.090352


In [12]:
print(len(np.arange(0.1,3.8,0.05)))

74


In [76]:
np.allclose(np.linalg.inv(matrix)@matrix,np.eye(16))
#matrix.shape

True

In [90]:
#check susy->the UBW commutes with the total supercharge
t,g=1,1

print('Qtot is',np.allclose(matrix@(Qplus(T,G)+Qminus(T,G)),(Qplus(T,G)+Qminus(T,G))@(matrix)))

Qtot is True


In [ ]:
######EXECUTION########
#specify the credentials when initializing QiskitRuntimeService, if they are not previously saved
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator")

with Session(service, backend=backend) as session:
    sampler = Sampler(session=session)
    job = sampler.run(qc, shots=16000)
    result = job.result()
renyi[k]=-np.log(result.quasi_dists[0][0]-result.quasi_dists[0][1]) 
print(k,T,renyi[k])